In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
from metpy.units import units
import matplotlib.pyplot as plt

In [ ]:
from agroservices import IPM 

In [ ]:
ipm = IPM()

In [ ]:
response=ipm.get_weatheradapter(
        endpoint='/weatheradapter/fmi/',
        credentials=None,
        weatherStationId=101104, 
        timeStart='2020-06-12', 
        timeEnd='2020-07-03',
        interval=3600, 
        parameters=[1002,3002]
        )
response

In [ ]:
times = pd.date_range(
        start=response['timeStart'].split('T')[0], 
        end=response['timeEnd'].split('T')[0], 
        freq="H",
        name="time")
parameter = response['weatherParameters']
lat=np.tile(response['locationWeatherData'][0]['latitude'],(1,505))

In [93]:
d_array = xr.DataArray(
    data=response['locationWeatherData'][0]['data'],
    coords={'time':times,
            'parameter':[str(el) for el in parameter],
            'latitude':response['locationWeatherData'][0]['latitude'],
            'longitude':response['locationWeatherData'][0]['longitude']
           },
    dims=['time','parameter']
)

# Attribute
# Data attribute
d_array.attrs['weatherRessource']= 'fmi'
d_array.attrs["stationId"]={"id":101104,"longitude":response['locationWeatherData'][0]['longitude'],"latitude":response['locationWeatherData'][0]['latitude']}
d_array.attrs["time"]={"timeStart":response['timeStart'], "timeEnd":response['timeStart'], "interval":3600,"timezone":"UTC"}
d_array.attrs['parameter']= response["weatherParameters"]



d_array.to_dataset(dim="parameter")

<xarray.Dataset>
Dimensions:    (time: 505)
Coordinates:
  * time       (time) datetime64[ns] 2020-06-11 ... 2020-07-02
    latitude   float64 60.81
    longitude  float64 23.5
Data variables:
    1002       (time) float64 16.2 13.7 11.5 9.9 8.7 ... 13.6 13.2 13.2 12.5
    3002       (time) float64 63.0 75.0 85.0 94.0 97.0 ... 99.0 98.0 99.0 100.0
Attributes:
    weatherRessource:  fmi
    stationId:         {'id': 101104, 'longitude': 23.49825, 'latitude': 60.8...
    time:              {'timeStart': '2020-06-11T21:00:00Z', 'timeEnd': '2020...
    parameter:         [1002, 3002]

<xarray.DataArray 'parameter' ()>
array('1002', dtype='<U4')
Coordinates:
    parameter  <U4 '1002'
    latitude   float64 60.81
    longitude  float64 23.5
{'id': 1002, 'name': 'Mean air temperature at 2m', 'description': None, 'unit': 'Celcius'}
<xarray.DataArray 'parameter' ()>
array('3002', dtype='<U4')
Coordinates:
    parameter  <U4 '3002'
    latitude   float64 60.81
    longitude  float64 23.5
{'id': 3002, 'name': 'Mean RH at 2m', 'description': None, 'unit': '%'}


In [ ]:
# add attribute
## coordinate attribute

ds.coords['latitude'].attrs['name']='latitude'
ds.coords['latitude'].attrs['unit']='degrees_north'

ds.coords['longitude'].attrs['name']='longitude'
ds.coords['longitude'].attrs['unit']='degrees_east'

## variable attribute
param = ipm.get_parameter()
p={str(item['id']): item for item in param if item['id'] in response['weatherParameters']}

for el in list(ds.data_vars):
    ds.data_vars[el].attrs=p[str(el)]

ds

In [60]:
param = ipm.get_parameter()
p=[item for item in param if item['id'] in response['weatherParameters']]


{'id': 1002,
 'name': 'Mean air temperature at 2m',
 'description': None,
 'unit': 'Celcius'}

In [ ]:
import metpy.calc as mpcalc
from metpy.units import units

In [ ]:
ds.data_vars[1002]

In [ ]:
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr

# Any import of metpy will activate the accessors
from metpy.cbook import get_test_data

#ds = xr.open_dataset(get_test_data('narr_example.nc', as_file_obj=False))
data_var = ds.metpy.parse_cf(1002)

In [ ]:

x = data_var.x
y = data_var.y
im_data = data_var.isel(time=0).sel(isobaric=1000.)

fig = plt.figure(figsize=(14, 14))
ax = fig.add_subplot(1, 1, 1, projection=data_var.metpy.cartopy_crs)

ax.imshow(im_data, extent=(x.min(), x.max(), y.min(), y.max()),
          cmap='RdBu', origin='lower' if y[0] < y[-1] else 'upper')
ax.coastlines(color='tab:green', resolution='10m')
ax.add_feature(cfeature.LAKES.with_scale('10m'), facecolor='none', edgecolor='tab:blue')
ax.add_feature(cfeature.RIVERS.with_scale('10m'), edgecolor='tab:blue')

plt.show()